# Resources for the Digital Encryption Standard (DES) questions

## Introduction
The DES questions are inspired by question 4.11 from Stallings. The question parts guide you through the application of a single round of the DES encryption algorithm. 

The algorithm is fully specified in appendix C of <a href="https://mmu.on.worldcat.org/oclc/1334132058" target="_blank">Stallings</a>. There you will find flowcharts, data tables and explanations of how the DES algorithm works. 

The questions will take you through the initial preparation of your message $m$ and key $k$, the calculation of the first sub-key $K_1$, and then the calculation of $L_1$ and $R_1$, the outputs of round 1 of DES. You are asked to maintain $m$, $k$ and the other intermediate outputs as *binary strings*, i.e. Python strings consisting made up of the characters `1` and `0`.  

To help you confirm the correctness of your work I have pre-computed the correct outputs of the final DES question for each student. If your final output matches with this then you can be confident that your answer is technically correct. If not, you must still have some unresolved errors in your implementation of DES. You can find these checking strings at <a href="./des.html" target="_blank">this link</a>.

In this notebook you will find 

* demonstrations of commands for basic string handling operations in Python,
* a demonstration of how the original keystring `k` is computed,
* Python definitions for lists defining the various transformation functions that form part of DES, as described in appendix C,
* a XOR Python function for performing the XOR of two binary strings.

 

## Working with strings in Python

*Some useful commands and techniques for working on the DES questions.*

There are many good online guides to working with strings in Python, for example the <a href="https://developers.google.com/edu/python/strings" target="_blank">Strings section</a> of <a href="https://developers.google.com/edu/python" target="_blank">Google's Python Class</a>. So consult this or other guides for further details. 

But your previous programming experience and the tips and commands in this notebook should be enough to get you through the DES questions. 

### String indexing and slicing

Indexing and slicing is how we can reference and use individual characters and substrings of a string. Indexing and slicing of strings in Python shares a lot of behaviour with indexing and slciing of arrays or lists.

Consider the following 64 bit binary string `k`. This is actually the key string that you will be using in answering the DES questions. Here we will use it to demonstrate some string manipulation techniques.

In [1]:
k = '0011000100110010001100110011010000110101001101100011011100111000'
k

'0011000100110010001100110011010000110101001101100011011100111000'

The command `len(k)` returns the length of the string, i.e. the number of characters in the string. 

In [2]:
len(k)

64

Individual characters of the string `k` can be referenced by `k[i]`, where `i` is the index of a character in the string. Remember that Python, like most languages, uses 0-based indexing of strings and arrays. So here are the first three characters of `k`

In [4]:
k[0]

'0'

In [5]:
k[1]

'0'

In [6]:
k[2]

'1'

Note that these commands return single character strings.

This indexing can be extended to refer to substrings within `k`. This is sometimes referred to as string *slicing*. For instance the following returns a ten character substring `s` of `k`, consisting of the bits from index position 3 through 12, i.e. the fourth through thirteenth bits of `k`. 

In [6]:
s = k[3:13]
s

'1000100110'

In [7]:
len(s)

10

### Transforming a string

Your work on the DES questions will require to perform various transformations of strings. 

As an example, let us transform `k` into a new string `h`, where the bit at index `i` of `h` is the bit at index `R[i]` of `k`, where `R` is an array of random indexes (with perhaps some repetitions) from the list $0, 1, 2, 3, \dots 63$.

In [7]:
from random import randint

In [8]:
R = [randint(1,64) for j in range(64)] 
# This creates R as an array of 64 random integers, selected from the 
# range 1 to 64. These will be interpreted as *plain English* index 
# index positions.

In [10]:
len(R)

64

In [12]:
h = ''.join(k[R[i]-1] for i in range(64))

In [13]:
k

'0011000100110010001100110011010000110101001101100011011100111000'

In [14]:
h

'1010010011001011100100011110111000110101000110100010001000101010'

*Inspect the array `R` and verify some of the entries of the string `h`.*

### Warning on array/string indexing

<p style="background-color:gold;">In the description of DES in Appendix C of the Stallings textbook, when referencing the position of a bit in a message block he uses index references beginning at 1, which is appropriate when speaking in plain English. This convention is also used in the various variables defined below that define the various transformations from Appendix C. So the left-most bit in a message block is at index 1, the next bit is at index 2 and so on. However, when referencing bit positions in binary strings in your Python commands you will need to remember that Python references positions in arrays, strings etc using indexing beginning at 0. So the leftmost bit in a binary string is at index 0, the second bit is at index 1, and so on. You will need to be careful when switching between plain language descriptions and Python code to avoid any *off-by-one* errors.</p>

<p style="background-color:gold;"><em>You should appreciate and understand how we have accounted for this in the command for applying the transformation represented by the array <code>R</code> to the string <code>k</code> above.</em></p>

## Computing the original keystring `k`

### Guidance

The message $m$ you will use is the 64 bit binary string representing the ASCII encoding of the eight digits of your 8-digit university ID number. 

The key $k$ you will use is the 64 bit binary string defined as

    k = '0011000100110010001100110011010000110101001101100011011100111000'
    
This string is the binary representation of the ASCII encoding of the digits in the string `'12345678'` which is calculated as shown in the Python block below. You should use similar commands to generate your initial message string $m$ that you will encrypt. 

In [15]:
k = '12345678'

# the command below redefines k as a list of the binary strings of the ASCII encodings of the 
# characters of the initial string k.
# bin(ord(x)) gives the python binary string representing the ASCII encoding of 
# the character x. Slicing this from index 2 onwards removes the `0b` binary indication code
# from the front of the string. zfill(8) fills in any leading zeros that were stripped 
# from bin(ord(x)) to give a standard 8 bit binary string.

k = [bin(ord(x))[2:].zfill(8) for x in k]

# the join method joins the eight 8-bit binary strings from the list k with the empty string, i.e. it 
# concatenates the eight 8-bit binary strings into a single 64-bit binary string. It does this by joining
# each of the strings with the empty string ''.

k = ''.join(x for x in k)
print(k)

0011000100110010001100110011010000110101001101100011011100111000


## Table data, as in Appendix C of the Stallings textbook

In [21]:
# Initial Permutation
IP=[
58, 50, 42, 34, 26, 18, 10, 2,
60, 52, 44, 36, 28, 20, 12, 4,
62, 54, 46, 38, 30, 22, 14, 6,
64, 56, 48, 40, 32, 24, 16, 8,
57, 49, 41, 33, 25, 17, 9, 1,
59, 51, 43, 35, 27, 19, 11, 3,
61, 53, 45, 37, 29, 21, 13, 5,
63, 55, 47, 39, 31, 23, 15, 7]

In [22]:
# Expansion permutation
E = [32,1,2,3,4,5,4,5,6,7,8,9,8,9,10,11,12,13,12,13,14,15,16,17, 
16,17,18,19,20,21,20,21,22,23,24,25,24,25,26,27,28,29,28,29,30,31,32,1]

In [23]:
#S-boxes, as a list of lists
SBOX = [
# Box-1
[
[14,4,13,1,2,15,11,8,3,10,6,12,5,9,0,7],
[0,15,7,4,14,2,13,1,10,6,12,11,9,5,3,8],
[4,1,14,8,13,6,2,11,15,12,9,7,3,10,5,0],
[15,12,8,2,4,9,1,7,5,11,3,14,10,0,6,13]
],
# Box-2

[
[15,1,8,14,6,11,3,4,9,7,2,13,12,0,5,10],
[3,13,4,7,15,2,8,14,12,0,1,10,6,9,11,5],
[0,14,7,11,10,4,13,1,5,8,12,6,9,3,2,15],
[13,8,10,1,3,15,4,2,11,6,7,12,0,5,14,9]
],

# Box-3

[
[10,0,9,14,6,3,15,5,1,13,12,7,11,4,2,8],
[13,7,0,9,3,4,6,10,2,8,5,14,12,11,15,1],
[13,6,4,9,8,15,3,0,11,1,2,12,5,10,14,7],
[1,10,13,0,6,9,8,7,4,15,14,3,11,5,2,12]

],

# Box-4
[
[7,13,14,3,0,6,9,10,1,2,8,5,11,12,4,15],
[13,8,11,5,6,15,0,3,4,7,2,12,1,10,14,9],
[10,6,9,0,12,11,7,13,15,1,3,14,5,2,8,4],
[3,15,0,6,10,1,13,8,9,4,5,11,12,7,2,14]
],

# Box-5
[
[2,12,4,1,7,10,11,6,8,5,3,15,13,0,14,9],
[14,11,2,12,4,7,13,1,5,0,15,10,3,9,8,6],
[4,2,1,11,10,13,7,8,15,9,12,5,6,3,0,14],
[11,8,12,7,1,14,2,13,6,15,0,9,10,4,5,3]
],
# Box-6

[
[12,1,10,15,9,2,6,8,0,13,3,4,14,7,5,11],
[10,15,4,2,7,12,9,5,6,1,13,14,0,11,3,8],
[9,14,15,5,2,8,12,3,7,0,4,10,1,13,11,6],
[4,3,2,12,9,5,15,10,11,14,1,7,6,0,8,13]

],
# Box-7
[
[4,11,2,14,15,0,8,13,3,12,9,7,5,10,6,1],
[13,0,11,7,4,9,1,10,14,3,5,12,2,15,8,6],
[1,4,11,13,12,3,7,14,10,15,6,8,0,5,9,2],
[6,11,13,8,1,4,10,7,9,5,0,15,14,2,3,12]
],
# Box-8

[
[13,2,8,4,6,15,11,1,10,9,3,14,5,0,12,7],
[1,15,13,8,10,3,7,4,12,5,6,11,0,14,9,2],
[7,11,4,1,9,12,14,2,0,6,10,13,15,3,5,8],
[2,1,14,7,4,10,8,13,15,12,9,0,3,5,6,11]
]

]

In [24]:
# Permutation P
P = [16,7,20,21,29,12,28,17,1,15,23,26,5,18,31,10,
2,8,24,14,32,27,3,9,19,13,30,6,22,11,4,25]

In [25]:
# Permuted choice 1, used in key generation
PC1 = [57,49,41,33,25,17,9,1,58,50,42,34,26,18,10,2,59,51,43,35,27,19,11,3,60,
52,44,36,63,55,47,39,31,23,15,7,62,54,46,38,30,22,14,6,61,53,45,37,29,21,13,5,28,20,12,4]

In [26]:
# Permuted choice 2, used in key generation
PC2 = [14,17,11,24,1,5,3,28,15,6,21,10,23,19,12,4,26,8,16,7,27,20,13,2,41,52,
31,37,47,55,30,40,51,45,33,48,44,49,39,56,34,53,46,42,50,36,29,32]

## Code for a XOR function

In [27]:
# Python function for carrying out the xor of two binary strings

def XOR(a,b):
    # a and b should be binary strings of equal length
    c = ''.join(str((int(x) + int(y))%2) for x,y in zip(a,b))
    return c

And here is an example of its usage. 

In [28]:
XOR('1100','0101')

'1001'